In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

# Create client

In [2]:
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama", # Ollama 不需要金鑰，但這裡隨便填一個字串即可，不然 SDK 會報錯
)

In [3]:

chat_completion = client.chat.completions.create(
    model="qwen3:8b",
    messages=[{"role": "user", "content": "Hello world"}]
    # ,stream=True
)

In [4]:
chat_completion.choices[0].message.content

'Hello! 😊 How can I assist you today?'

# Create Agent

In [17]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})
    def __call__(self, user_input):
        self.messages.append({"role": "user", "content": user_input})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    def execute(self):
        chat_completion = client.chat.completions.create(
            model="qwen3:8b",
            temperature=0,
            messages=self.messages,
            stop=["Observation:"]
        )
        return chat_completion.choices[0].message.content

In [50]:
test_agent = Agent(system="You are a helpful assistant.")
test_agent("What is the capital of France?")

'The capital of France is **Paris**. It is a major cultural, political, and economic center in Europe, known for landmarks such as the Eiffel Tower, the Louvre Museum, and the Seine River.'

# ReAct

In [38]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number.
CRITICAL RULE: You MUST NOT perform any math in your head. Even for simple additions like 1+1, you MUST use the calculate action. If you output a math result without using the calculate action first, you will fail.

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [39]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [40]:
abot = Agent(prompt)

In [20]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a toy poodle using the average_dog_weight function.
Action: average_dog_weight: Toy Poodle
PAUSE




In [21]:
result = average_dog_weight("Toy Poodle")
print(result)

a toy poodles average weight is 7 lbs


In [22]:
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

'Answer: A toy poodle weighs 7 lbs.'

In [23]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [41]:
abot = Agent(prompt)
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then calculate their combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE\n\n'

In [42]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [43]:
abot(next_prompt)
print(abot.messages[-1]['content'])

Thought: Now I need to find the average weight of a Scottish Terrier and then calculate the combined weight of both dogs.
Action: average_dog_weight: Scottish Terrier
PAUSE

</think>




In [44]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [45]:
abot(next_prompt)
print(abot.messages[-1]['content'])

Action: calculate: 37 + 20
PAUSE




In [47]:
next_prompt = "Observation: {}".format(calculate("37 + 20"))
print(next_prompt)

Observation: 57


In [48]:
abot(next_prompt)
print(abot.messages[-1]['content'])

Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.


In [49]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number.\nCRITICAL RULE: You MUST NOT perform any math in your head. Even for simple additions like 1+1, you MUST use the calculate action. If you output a math result without using the calculate action first, you will fail.\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with th

# Add loop

In [51]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [52]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i+=1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.splitlines()
            if action_re.match(a)
        ]
        if actions:
            action, actions_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, actions_input))
            print(" -- running {} {}".format(action, actions_input))
            observation = known_actions[action](actions_input)
            print(("Observation: {}".format(observation)))
            next_prompt = "Observation: {}".format(observation)
        else:
            return 

In [53]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then calculate their combined weight.
Action: average_dog_weight: Border Collie
PAUSE


 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier and then calculate the combined weight of both dogs.
Action: average_dog_weight: Scottish Terrier
PAUSE

</think>


 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Action: calculate: 37 + 20
PAUSE


 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.
